## Question 3

## Bootstrapping

Bootstrapping allows assigning measures of accuracy (defined in terms of bias, variance, confidence intervals, prediction error or some other such measure) to sample estimates. This technique allows estimation of the sampling distribution of almost any statistic using random sampling methods.

It is always inappropriate to determine the accuracy of the dataset with one sample. The accuracy can be better determined by tuning the given dataset into different samples (or) using different training data sets , and then obtaining the average of those samples.

In [1]:
#reading train data, printing few lines of training data
task1b_train_data <- read.csv("Task1B_train.csv")
head(task1b_train_data)
#displaying the no.of dimensions of training data
dim(task1b_train_data)

x1,x2,x3,x4,y
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1991.496,1628.75,1678.1,1772.8,2443.6
1991.504,1606.51,1678.6,1718.0,2448.2
1991.512,1618.16,1686.6,1723.1,2484.7
1991.519,1630.75,1682.9,1734.5,2487.9
1991.527,1635.47,1697.5,1754.0,2510.5
1991.535,1647.84,1723.8,1759.8,2532.5


[1] 930   5

In [2]:
#split the train data into the data without the labels column and displaying a few lines of it
train.data <- task1b_train_data[,-5]
head(train.data)
#displaying the no.of dimensions of training data
dim(train.data)

x1,x2,x3,x4
<dbl>,<dbl>,<dbl>,<dbl>
1991.496,1628.75,1678.1,1772.8
1991.504,1606.51,1678.6,1718.0
1991.512,1618.16,1686.6,1723.1
1991.519,1630.75,1682.9,1734.5
1991.527,1635.47,1697.5,1754.0
1991.535,1647.84,1723.8,1759.8


[1] 930   4

In [3]:
#getting the label of train data and displaying the few lines of it
train.label <- task1b_train_data[,5]
head(train.label)

[1] 2443.6 2448.2 2484.7 2487.9 2510.5 2532.5

In [4]:
task1b_test_data <- read.csv("Task1B_test.csv")
head(task1b_test_data)

x1,x2,x3,x4,y
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1991.500,1613.63,1688.5,1750.5,2460.2
1991.508,1621.04,1684.1,1708.1,2470.4
1991.515,1610.61,1671.6,1714.3,2466.8
1991.523,1640.17,1703.6,1757.4,2508.4
1991.531,1645.89,1716.3,1754.3,2497.4
1991.538,1638.35,1730.5,1755.5,2556.8


In [5]:
#reading test data, printing few lines of testing data
test.data <- task1b_test_data[,-5]
head(test.data)
#displaying the no.of dimensions of testing data
dim(test.data)

x1,x2,x3,x4
<dbl>,<dbl>,<dbl>,<dbl>
1991.500,1613.63,1688.5,1750.5
1991.508,1621.04,1684.1,1708.1
1991.515,1610.61,1671.6,1714.3
1991.523,1640.17,1703.6,1757.4
1991.531,1645.89,1716.3,1754.3
1991.538,1638.35,1730.5,1755.5


[1] 930   4

In [7]:
#getting the label of train data and displaying the few lines of it
test.label <- task1b_test_data[,5]
head(test.label)

[1] 2460.2 2470.4 2466.8 2508.4 2497.4 2556.8

In [8]:
boot <- function (original.size=100, sample.size=original.size, times=100){
    indx <- matrix(nrow=times, ncol=sample.size)
    for (t in 1:times){
        indx[t, ] <- sample(x=original.size, size=sample.size, replace = TRUE)
    }
    return(indx)
}

In [9]:
#using knn classification algorithm, for each test data point, we will predict the test label based on the nearest train data points
knnregressor <- function(train.data, train.label, test.data, K = 3, distance = 'manhattan'){
    
    #getting the length of train and test data points
    train.len <- nrow(train.data)
    test.len <- nrow(test.data)
    
    #for each test data point, we will calculate the distance of that particular test data point with respect to all training data points, and then select
    #an appropriate portion of it.
    dist <- as.matrix(dist(rbind(test.data, train.data), method = distance))[1:test.len, (test.len+1):(test.len+train.len)]
    
    test_label_predicted <- c()
    
    #we will traverse through via for loop for all the test data points, and for each test data point, the portion of train data points obtained are sorted 
    #based on the distances and the average of those train label points is estimated as the label for test data point.
    for (i in 1:test.len){
        nn <- as.data.frame(sort(dist[i,], index.return = TRUE))[1:K,2]
        test_label_predicted[i] <- mean(train.label[nn])
    }
    return (test_label_predicted)
}

## Question 3.2 

Apply your
bootstrapping for KNN regression with times = 50 (the number of subsets),
size = 20 (the size of each subset), and change K=1,.., 15 (the neighbourhood
size). Now create a boxplot where the x-axis is K, and the y-axis is the
average error (and the uncertainty around it) corresponding to each K. 

In [19]:
K <- 15   #neighbourhood size
L <- 50   #no.of subsets
N <- 20   #size of each subset

#with the above mentioned hyper parameters, boot function is called which returns a set of samples in the matrix form, where each sample represents one row
#and contain sample indexes in each row
boot.indx <- boot(nrow(train.data), N, L)

#creating an empty dataframe
error_k_l <- data.frame('K'=1:K, 'L'=1:L, 'test'=rep(0,L*K))

#with respect to each 'k' value, traverse through the 'L' values, get the indexes of the samples and then pass it through the knnregressor function 
for (k in 1: K){
    for (l in 1:L){
        i <- (k-1)*L+l
        indx <- boot.indx[l,]
        error_k_l[i,'K'] <- k
        error_k_l[i,'L'] <- l
        error_k_l[i,'test'] <-  sqrt(sum((knnregressor(train.data[indx, ], train.label[indx], test.data, K=k)-test.label)**2)/nrow(test.data))

    } 
    
}
head(error_k_l)

K,L,test
<int>,<int>,<dbl>
1,1,152.0744
1,2,165.2456
1,3,183.6514
1,4,166.5955
1,5,277.7868
1,6,144.4244


### create a boxplot where the x-axis is K, and the y-axis is the average error (and the uncertainty around it) corresponding to each k
 

In [21]:
#importing the required libraries
library(reshape2)
library(ggplot2)
miss_k_l <- melt(error_k_l, id=c('K', 'L')) # reshape for visualization
names(miss_k_l) <- c('K', 'L', 'type', 'miss')
#ploting the graph with required parameters, i.e, k versus errors
ggplot(data=miss_k_l[miss_k_l$type=='test',], aes(factor(K), miss,fill=type)) + geom_boxplot(outlier.shape = NA)  + 
    scale_color_discrete(guide = guide_legend(title = NULL)) + 
    ggtitle('Missclassifcation vs. K (Box Plot)') + theme_minimal()
# ignore the warnings (because of ignoring outliers)
options(warn=-1)

### Question 3.3 How does the test error and its uncertainty behave as K increases? 

1.With increase of k value, the number of missclassifications i.e, test errors increases. 

2.The median proportion of test errors with increase of k value increases.

3.Uncertainty is defined as the size of the boxplot. If we observe the above box-plot, uncertainity of test errors increased until k = 10, and then slightly decreasing.

4.When k is less, the testing data suffers from over-fitting but when k is large it suffers from under-fitting.

## Question 3.4

Apply your
bootstrapping for KNN regression with K= 5(the neighbourhood size),
times = 50 (the number of subsets), and change sizes = 5, 10, 15,..., 75 (the
size of each subset).

In [27]:
K <- 5  #neighbourhood size
L <- 50 #no.of subsets
#creating an empty dataframe
miss_new <- data.frame('L'= 1:750, 'test'= 1:750)
i <- 1  #iteration counter

#with the above mentioned hyper parameters for each size, with a given k , l value we are calling the boot function, obtaining the respective indexes and
#then calling knnregressor function and finally obtaining the errors of test data points.

for (n in seq(5,75,5)){
    boot.indx <- boot(nrow(train.data), n, L)
    for (l in 1:50){
        indx <- boot.indx[l,]
        miss_new[i,'L'] <- n
        miss_new[i,'test'] <-  sqrt(sum((knnregressor(train.data[indx, ], train.label[indx], test.data, K = K)-test.label)**2)/nrow(test.data))
        i <- i + 1
    }
}

In [28]:
miss_new.m <- melt(miss_new, id=c('L')) # reshape for visualization

names(miss_new.m) <- c('L', 'type', 'miss') #column names for the reshaped data frame

### create a boxplot where the x-axis is sizes, and the y-axis is the average error (and the uncertainty around it) corresponding to each k

In [30]:
ggplot(data=miss_new.m[miss_new.m$type=='test',], aes(factor(L), miss,fill=type)) + geom_boxplot(outlier.shape = NA) +
       scale_color_discrete(guide = guide_legend(title = NULL)) +  ggtitle('Missclassifcation vs. L (Box Plot)') + theme_minimal()

# ignore the warnings (because of ignoring outliers)
options(warn=-1)


### How does the test error and its uncertainty behave as the size of each subset in bootstrapping increases? 

1.As the size of each subset is increasing, the test error rate is decreasing.

2.When comes to the uncertainty, with increase of L i.e size, the uncertainity increases and then slightly decreases. Uncertainty is refered as the size of boxplot.

3.When the uncertainty is more, the model suffers from under-fitting and when uncertainty is less, the model suffers from over-fitting.

4.At k = 15, the model suffers from under-fitting, as uncertainty is more.